In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, add_messages, START, END
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import MessagesState
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langchain.chat_models import init_chat_model

llm = init_chat_model("ollama:qwen2.5:32b", temperature=0)

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

In [3]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

In [7]:
tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

In [13]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # removed assert since we have two tools being called later on
    return {"messages": [message]}

In [8]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # removed assert since we have two tools being called
    # try only one tool call?
    # maybe the second tool call will be updated after the first?
    one_tool_call = message.tool_calls[0]
    message.tool_calls = [one_tool_call]
    return {"messages": [message]}

In [ ]:
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # check if there is a tool call
    if len(message.tool_calls) > 0:
        one_tool_call = message.tool_calls[0]
        # modify the tool call to only have one
        message.tool_calls = [one_tool_call]
    return {"messages": [message]}

In [9]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [10]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (2f8f1856-3f85-4fb1-87cd-ef0c42701735)
 Call ID: 2f8f1856-3f85-4fb1-87cd-ef0c42701735
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph release date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://pypi.org/project/langgraph/", "title": "langgraph·PyPI", "content": "langgraph 0.4.8 Image 5: LangGraph Logo Install LangGraph: from langgraph.prebuilt import create_react_agent Or, to learn how to build an agent workflow with a customizable architecture, long-term memory, and other complex task handling, see the LangGraph basics tutorials.

In [11]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

The search results do not provide a specific release date for LangGraph. However, the latest version mentioned on PyPI is `0.4.8`, and there's an older version `0.3.0` that was yanked (removed) due to missing dependencies. The exact release dates are not specified in these sources.

Given this information, I will request human assistance for a more accurate review.
Tool Calls:
  human_assistance (e85729bf-e4ae-4913-82fd-98648c0ee958)
 Call ID: e85729bf-e4ae-4913-82fd-98648c0ee958
  Args:
    birthday: 
    name: LangGraph Release Date Review
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================
Tool Calls:
  human_assistance (2d7588d6-2522-48a9-930d-047cce060972)
 Call ID: 2d7588d6-2522-48a9-930d-047cce06097

In [12]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

In [13]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06270a-4926-66cf-8006-dda53786985d'}}

In [14]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}

In [15]:
human_command = Command(
    resume={
        "correct": "yes",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (2d7588d6-2522-48a9-930d-047cce060972)
 Call ID: 2d7588d6-2522-48a9-930d-047cce060972
  Args:
    birthday: Jan 17, 2024
    name: LangGraph
================================= Tool Message =================================
Name: human_assistance

Correct


IndexError: list index out of range